In [ ]:
# src/portafolios/data/local_loader.py
import pandas as pd
from pathlib import Path
from typing import Optional, Iterable

def local_loader(
    tickers: Optional[Iterable[str]] = None,
    start: Optional[str] = None,
    end: Optional[str] = None,
    path: str = "data_clean_stock_data.csv"
) -> pd.DataFrame:
    """
    Carga un DataFrame local (ya descargado de yfinance)
    y devuelve solo las columnas de cierre por ticker.
    
    Parámetros
    ----------
    tickers : lista opcional de tickers a incluir.
    start, end : rango de fechas.
    path : ruta al archivo CSV local.

    Retorna
    -------
    DataFrame con índices datetime y columnas = tickers (solo precios de cierre).
    """
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"No se encontró el archivo: {path}")

    # --- leer archivo
    df = pd.read_csv(path, header=[0,1], index_col=0)
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    # --- elegir solo columnas de cierre
    close_cols = []
    for t, f in df.columns:
        if f in ["Adj Close", "Close"]:
            close_cols.append((t, f))
    df_close = df[close_cols].copy()
    df_close.columns = df_close.columns.get_level_values(0)  # solo los tickers

    # --- filtrar tickers, fechas y frecuencia
    if tickers is not None:
        df_close = df_close.loc[:, [t for t in tickers if t in df_close.columns]]
    if start:
        df_close = df_close.loc[df_close.index >= pd.to_datetime(start)]
    if end:
        df_close = df_close.loc[df_close.index <= pd.to_datetime(end)]

    # --- remuestrear si hace falta
    if freq:
        df_close = df_close.asfreq(freq, method="pad")

    return df_close


def basic_clean(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocesamiento mínimo: elimina duplicados y forward-fill de valores faltantes.
    """
    df = df.sort_index().drop_duplicates()
    df = df.ffill().bfill()
    return df
